In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt
import os

In [ ]:
from cmdstanpy import install_cmdstan
install_cmdstan()

In [ ]:
# Set random seed
np.random.seed(42)

# Load preprocessed data
train_data = pd.read_csv('processed_train_data.csv')
test_data = pd.read_csv('processed_test_data.csv')

# Check if data is empty
if len(train_data) == 0:
    raise ValueError("Train data is empty. Check preprocessing step.")
if len(test_data) == 0:
    print("Warning: Test data is empty. Predictions will be skipped.")

# Define features (exclude quality flags and season)
categorical_cols = ['is_raining', 'day_of_week', 'month']
numerical_cols = ['passenger_count_lag1', 'temp_7d_mean', 'max_temp', 'Q_TG',
                  'precipitation', 'snow_depth', 'global_radiation', 'pressure',
                  'humidity', 'cloud_cover']
features = numerical_cols + categorical_cols
target = 'passenger_count'

In [ ]:
# Prepare training and test sets with polynomial and interaction terms
X_train = train_data[features].copy()
X_test = test_data[features].copy() if len(test_data) > 0 else pd.DataFrame(columns=X_train.columns)
y_train = train_data[target]
y_test = test_data[target] if len(test_data) > 0 else pd.Series()

In [ ]:
# Add polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X_train[numerical_cols])
poly_feature_names = poly.get_feature_names_out(numerical_cols)
X_train_poly = pd.DataFrame(poly_features, columns=poly_feature_names, index=X_train.index)
X_train = pd.concat([X_train.drop(columns=numerical_cols), X_train_poly], axis=1)

if len(X_test) > 0:
    poly_features_test = poly.transform(X_test[numerical_cols])
    X_test_poly = pd.DataFrame(poly_features_test, columns=poly_feature_names, index=X_test.index)
    X_test = pd.concat([X_test.drop(columns=numerical_cols), X_test_poly], axis=1)

In [ ]:
# Add interaction term (passenger_count_lag1 * day_of_week effect)
dow_cols = [col for col in X_train.columns if col.startswith('day_of_week_')]
for dow_col in dow_cols:
    X_train[f'lag_dow_{dow_col}'] = X_train['passenger_count_lag1'] * X_train[dow_col]
    if len(X_test) > 0:
        X_test[f'lag_dow_{dow_col}'] = X_test['passenger_count_lag1'] * X_test[dow_col]
numerical_cols = list(X_train_poly.columns) + [f'lag_dow_{col}' for col in dow_cols]

In [ ]:
# Preprocess categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),
        ('num', 'passthrough', numerical_cols)
    ])
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test) if len(X_test) > 0 else np.array([])

In [ ]:
# Get feature names
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
feature_names = list(cat_feature_names) + numerical_cols
dow_indices = [i for i, name in enumerate(feature_names) if name.startswith('day_of_week_')]

In [ ]:
# Prior predictive checks
n_sim = 1000
beta0_sim = np.random.normal(0, 5, n_sim)
beta_sim = np.random.normal(0, 5, (n_sim, len(feature_names)))
mu_dow_sim = np.random.normal(0, 2, n_sim)
sigma_dow_sim = np.abs(np.random.standard_cauchy(n_sim))  # Relaxed scale
beta_dow_sim = np.array([np.random.normal(mu_dow_sim, 0.25) for _ in range(len(dow_indices))]).T  # Relaxed prior
sigma_sim = np.abs(np.random.standard_cauchy(n_sim) * 2.5)
print("Prior Parameter Checks:")
print(f"beta0 range: [{beta0_sim.min():.2f}, {beta0_sim.max():.2f}]")
print(f"beta range: [{beta_sim.min():.2f}, {beta_sim.max():.2f}]")
print(f"mu_dow range: [{mu_dow_sim.min():.2f}, {mu_dow_sim.max():.2f}]")
print(f"sigma_dow range: [{sigma_dow_sim.min():.2f}, {sigma_dow_sim.max():.2f}]")
print(f"beta_dow range: [{beta_dow_sim.min():.2f}, {beta_dow_sim.max():.2f}]")
print(f"sigma range: [{sigma_sim.min():.2f}, {sigma_sim.max():.2f}]")

In [ ]:
# Prior predictive measurements
if len(X_train_transformed) > 0:
    mu_sim = beta0_sim + X_train_transformed[:n_sim] @ beta_sim.T
    y_sim = np.array([np.random.normal(mu_sim[i], sigma_sim[i]) for i in range(n_sim)])
    print(f"Prior Predictive Measurements range: [{y_sim.min():.2f}, {y_sim.max():.2f}]")
    plt.hist(y_sim.flatten(), bins=50, density=True)
    plt.title('Prior Predictive Distribution of Passenger Counts (Model 2)')
    plt.xlabel('Normalized Passenger Count')
    plt.ylabel('Density')
    plt.savefig('prior_predictive_measurements_model2.png')
    plt.show()

In [ ]:
# Prepare Stan data
stan_data = {
    'N': len(X_train_transformed),
    'K': len(feature_names),
    'K_dow': len(dow_indices),
    'dow_indices': [i+1 for i in dow_indices],  # Stan uses 1-based indexing
    'X': X_train_transformed,
    'y': y_train.values,
    'N_new': len(X_test_transformed) if len(X_test) > 0 else 0,
    'X_new': X_test_transformed if len(X_test) > 0 else np.zeros((0, len(feature_names)))
}

In [ ]:
# Define Stan model with relaxed priors
stan_code = """
data {
  int<lower=0> N;          // Number of training samples
  int<lower=0> K;          // Number of features
  int<lower=0> K_dow;      // Number of day_of_week features
  array[K_dow] int<lower=1,upper=K> dow_indices; // Indices of day_of_week features - Corrected syntax
  matrix[N, K] X;          // Feature matrix
  vector[N] y;             // Target variable
  int<lower=0> N_new;      // Number of test samples
  matrix[N_new, K] X_new;  // Test feature matrix
}
parameters {
  real beta0;              // Intercept
  vector[K] beta;          // Feature coefficients
  real mu_dow;             // Mean of day_of_week coefficients
  real<lower=0> sigma_dow; // SD of day_of_week coefficients
  real<lower=0> sigma;     // Noise SD
}
model {
  vector[N] mu;
  // Priors
  beta0 ~ normal(0, 5);
  for (k in 1:K) {
    // Note: Stan arrays are 1-indexed, but the loop is 1 to K.
    // dow_indices contains 1-based indices from Python's 0-based.
    // The 'in' operator works with 1-based indices as expected here.

    // Stan does not support 'in' operator directly for this check.
    // Use a loop to check if k is in dow_indices
    int is_dow = 0; // Flag to check if k is a day_of_week index
    for (i in 1:K_dow) {
      if (k == dow_indices[i]) {
        is_dow = 1;
        break; // Exit inner loop once a match is found
      }
    }

    if (is_dow) {
      beta[k] ~ normal(mu_dow, 0.25);  // Relaxed prior
    } else {
      beta[k] ~ normal(0, 5);
    }
  }
  mu_dow ~ normal(0, 2);
  sigma_dow ~ cauchy(0, 1);  // Relaxed scale
  sigma ~ cauchy(0, 2.5);

  // Likelihood
  for (n in 1:N) {
    mu[n] = beta0 + dot_product(X[n], beta);
  }
  y ~ normal(mu, sigma);
}
generated quantities {
  vector[N_new] y_pred;
  for (n in 1:N_new) {
    y_pred[n] = normal_rng(dot_product(X_new[n], beta) + beta0, sigma);
  }
}
"""

# Save Stan model
with open('hierarchical_regression_improved.stan', 'w') as f:
    f.write(stan_code)

In [ ]:
# Compile and fit model with optimized sampling
model = CmdStanModel(stan_file='hierarchical_regression_improved.stan')
fit = model.sample(data=stan_data, chains=4, iter_sampling=2000, iter_warmup=500, adapt_delta=0.9, max_treedepth=8, seed=42)

In [ ]:
# Check sampling diagnostics
summary = fit.summary()
rhat = summary['R_hat'].max() if 'R_hat' in summary.columns else float('nan')
n_eff = summary['n_eff'].min() if 'n_eff' in summary.columns else None
print(f"Sampling Diagnostics: Max R-hat = {rhat:.4f}, Min N_Eff = {n_eff if n_eff is not None else 'N/A'}")
if rhat > 1.1 or (n_eff is not None and n_eff < 100):
    print("Warning: Sampling issues detected. Consider further increasing iter_sampling or adapt_delta.")

In [ ]:
# Posterior predictive analysis
if len(X_test_transformed) > 0:
    y_pred = fit.stan_variable('y_pred').mean(axis=0)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Posterior Predictive Metrics (Model 2 - Optimized):")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    # Check consistency
    y_pred_samples = fit.stan_variable('y_pred')
    credible_intervals = np.percentile(y_pred_samples, [2.5, 97.5], axis=0)
    within_ci = np.mean((y_test >= credible_intervals[0]) & (y_test <= credible_intervals[1]))
    print(f"Proportion of test data within 95% credible intervals: {within_ci:.2f}")
    if within_ci < 0.9:
        print("Warning: Less than 90% of test data within credible intervals. Consider non-linear effects or additional features.")

In [ ]:
# Parameter marginal distributions
beta_samples = fit.stan_variable('beta')
beta0_samples = fit.stan_variable('beta0')
sigma_samples = fit.stan_variable('sigma')
mu_dow_samples = fit.stan_variable('mu_dow')
sigma_dow_samples = fit.stan_variable('sigma_dow')
print("\nParameter Summaries (Model 2 - Optimized):")
for i, name in enumerate(feature_names):
    mean, std = beta_samples[:, i].mean(), beta_samples[:, i].std()
    ci = np.percentile(beta_samples[:, i], [2.5, 97.5])
    print(f"{name}: Mean = {mean:.4f}, SD = {std:.4f}, 95% CI = [{ci[0]:.4f}, {ci[1]:.4f}]")
print(f"beta0: Mean = {beta0_samples.mean():.4f}, SD = {beta0_samples.std():.4f}, 95% CI = [{np.percentile(beta0_samples, 2.5):.4f}, {np.percentile(beta0_samples, 97.5):.4f}]")
print(f"mu_dow: Mean = {mu_dow_samples.mean():.4f}, SD = {mu_dow_samples.std():.4f}, 95% CI = [{np.percentile(mu_dow_samples, 2.5):.4f}, {np.percentile(mu_dow_samples, 97.5):.4f}]")
print(f"sigma_dow: Mean = {sigma_dow_samples.mean():.4f}, SD = {sigma_dow_samples.std():.4f}, 95% CI = [{np.percentile(sigma_dow_samples, 2.5):.4f}, {np.percentile(sigma_dow_samples, 97.5):.4f}]")
print(f"sigma: Mean = {sigma_samples.mean():.4f}, SD = {sigma_samples.std():.4f}, 95% CI = [{np.percentile(sigma_samples, 2.5):.4f}, {np.percentile(sigma_samples, 97.5):.4f}]")

In [ ]:
# Visualization: Actual vs. Predicted
if len(test_data) > 0 and 'date' in test_data.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(test_data['date'], y_test, label='Actual', color='blue')
    plt.plot(test_data['date'], y_pred, label='Predicted', color='red', alpha=0.7)
    plt.title('Actual vs. Predicted Passenger Counts (Model 2 - Optimized)')
    plt.xlabel('Date')
    plt.ylabel('Normalized Passenger Count')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('actual_vs_predicted_model2.png')
    plt.show()
    print("Visualization saved as 'actual_vs_predicted_model2.png'")

In [ ]:
# Save predictions
if len(test_data) > 0:
    test_data['predicted_passenger_count'] = y_pred
    test_data.to_csv('predictions_2023_model2_optimized.csv', index=False)
    print("Predictions saved as 'predictions_2023_model2_optimized.csv'")